In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

# Encode the target variable y_train
label_encoder = LabelEncoder()


# Load the dataset
data = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\preprocessing\rna_sequence_features.csv")

# Separate features (X) and target variable (y)
X = data.drop(columns=['RNA_Name'])
y = data['RNA_Name']

y=label_encoder.fit_transform(y)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize or scale the features if necessary (optional)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



# Train Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Train SVM classifier
# svm_classifier = SVC(kernel='linear', random_state=42)
# svm_classifier.fit(X_train_scaled, y_train)

# Train XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)
xgb_classifier.fit(X_train_scaled, y_train)

# Train LightGBM classifier
lgbm_classifier = lgb.LGBMClassifier(random_state=42)
lgbm_classifier.fit(X_train_scaled, y_train)

# Evaluate classifiers
rf_accuracy = rf_classifier.score(X_test_scaled, y_test)
# svm_accuracy = svm_classifier.score(X_test_scaled, y_test)
xgb_accuracy = xgb_classifier.score(X_test_scaled, y_test)
lgbm_accuracy = lgbm_classifier.score(X_test_scaled, y_test)

# Display the accuracies
print("Random Forest Accuracy:", rf_accuracy)
# print("SVM Accuracy:", svm_accuracy)
print("XGBoost Accuracy:", xgb_accuracy)
print("LightGBM Accuracy:", lgbm_accuracy)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3951
[LightGBM] [Info] Number of data points in the train set: 4550, number of used features: 87
[LightGBM] [Info] Start training from score -2.553586
[LightGBM] [Info] Start training from score -2.495956
[LightGBM] [Info] Start training from score -2.514800
[LightGBM] [Info] Start training from score -2.534005
[LightGBM] [Info] Start training from score -2.989161
[LightGBM] [Info] Start training from score -2.570680
[LightGBM] [Info] Start training from score -2.528480
[LightGBM] [Info] Start training from score -2.490637
[LightGBM] [Info] Start training from score -2.559251
[LightGBM] [Info] Start training from score -2.514800
[LightGBM] [Info] Start training from score -2.545147
[LightGBM] [Info] Start training from score -2.562096
[LightGBM] [Info] Start training from score -2.573558
Random For

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from hyperopt import hp, tpe, Trials, fmin
import os

# Load the dataset
data = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\rna_sequence_features.csv")

# Split features and labels
X = data.iloc[:, :-2].values  # Features
y = data.iloc[:, -1].values   # Labels (assuming they are in the last column)

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for CNN input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))  # Calculate the number of classes dynamically
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Define hyperparameter search space
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
}

# Define objective function
def objective(params):
    # Build the CNN model
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(params['dropout_rate']))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(params['dropout_rate']))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model with categorical crossentropy loss
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with one-hot encoded labels and early stopping
    model.fit(X_train, y_train_one_hot, epochs=25, batch_size=64, verbose=0, validation_data=(X_test, y_test_one_hot), callbacks=[early_stopping])

    # Evaluate the model
    loss, _ = model.evaluate(X_test, y_test_one_hot, verbose=0)
    
    # Return negative accuracy (minimize negative accuracy = maximize accuracy)
    return -loss

# Perform Bayesian Optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

print("Best hyperparameters:", best)

# Rebuild the best model with the best hyperparameters
best_model = Sequential()
best_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
best_model.add(MaxPooling1D(pool_size=2))
best_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
best_model.add(MaxPooling1D(pool_size=2))
best_model.add(Flatten())
best_model.add(Dense(256, activation='relu'))
best_model.add(Dropout(best['dropout_rate']))
best_model.add(Dense(128, activation='relu'))
best_model.add(Dropout(best['dropout_rate']))
best_model.add(Dense(num_classes, activation='softmax'))

# Compile the best model with categorical crossentropy loss
best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the best model with one-hot encoded labels and early stopping
best_model.fit(X_train, y_train_one_hot, epochs=25, batch_size=64, validation_data=(X_test, y_test_one_hot), callbacks=[early_stopping])

# Save the weights of the best model
best_model.save_weights('best_model_weights.h5')

# Evaluate the best model with the test data
test_loss, test_accuracy = best_model.evaluate(X_test, y_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 10/10 [00:19<00:00,  1.93s/trial, best loss: -1.426284909248352]
Best hyperparameters: {'dropout_rate': 0.02976027759400396, 'learning_rate': 0.0017931008791918584}
Epoch 1/25
8/8 [==============================] - 1s 28ms/step - loss: 2.4336 - accuracy: 0.1663 - val_loss: 2.2980 - val_accuracy: 0.3150
Epoch 2/25
8/8 [==============================] - 0s 8ms/step - loss: 2.1373 - accuracy: 0.3347 - val_loss: 2.0020 - val_accuracy: 0.3622
Epoch 3/25
8/8 [==============================] - 0s 9ms/step - loss: 1.8794 - accuracy: 0.4059 - val_loss: 1.8757 - val_accuracy: 0.2992
Epoch 4/25
8/8 [==============================] - 0s 7ms/step - loss: 1.6768 - accuracy: 0.4356 - val_loss: 1.6380 - val_accuracy: 0.4882
Epoch 5/25
8/8 [==============================] - 0s 9ms/step - loss: 1.5025 - accuracy: 0.5228 - val_loss: 1.5828 - val_accuracy: 0.4646
Epoch 6/25
8/8 [==============================] - 0s 10ms/step - loss: 1.3895 - accuracy: 0.5366 - val_loss: 1.5901 - val_accur

# last Hope dataset

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
import pandas as pd
data = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\preprocessing\features.csv')

# Separate features and target variable
X = data.drop(['RNA_Name'], axis=1)  # Assuming 'RNA_Name' is the target variable
y = data['RNA_Name']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.30755711775043937


In [12]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import numpy as np

# Define Lasso model
lasso = LassoCV(cv=5)

# Define feature selection
feature_selection = SelectFromModel(lasso)

# Define Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define LightGBM classifier
lgb_classifier = LGBMClassifier(n_estimators=100, random_state=42)

# Create a pipeline with Random Forest
pipeline_rf = Pipeline([('feature_selection', feature_selection),
                        ('random_forest', rf_classifier)])

# Create a pipeline with LightGBM
pipeline_lgb = Pipeline([('feature_selection', feature_selection),
                         ('lightgbm', lgb_classifier)])

# Train and test the Random Forest pipeline
pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# Train and test the LightGBM pipeline
pipeline_lgb.fit(X_train, y_train)
y_pred_lgb = pipeline_lgb.predict(X_test)
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy:", accuracy_lgb)


Random Forest Accuracy: 0.5861159929701231
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 4549, number of used features: 1
[LightGBM] [Info] Start training from score -7.324050
[LightGBM] [Info] Start training from score -7.729516
[LightGBM] [Info] Start training from score -5.426930
[LightGBM] [Info] Start training from score -8.422663
[LightGBM] [Info] Start training from score -7.036368
[LightGBM] [Info] Start training from score -7.324050
[LightGBM] [Info] Start training from score -7.729516
[LightGBM] [Info] Start training from score -8.422663
[LightGBM] [Info] Start training from score -7.324050
[LightGBM] [Info] Start training from score -6.813225
[LightGBM] [Info] Start training from score -6.343221
[LightGBM] [Info] Start training from score -6.630903
[LightGBM] [Info] Start

# New Hope data

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

# Encode the target variable y_train
label_encoder = LabelEncoder()


# Load the dataset
data = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\extracted_features_6c.csv")
tdata = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\extracted_features_6c_test.csv")
# data = data.drop(data[data['RNA_Name'] == 'HACA-box'].index)
# data = data.drop(data[data['RNA_Name'] == 'CD-box'].index)
# data = data.drop(data[data['RNA_Name'] == 'Leader'].index)
# data = data.drop(data[data['RNA_Name'] == 'HACA-box'].index)

# Separate features (X) and target variable (y)
# data.drop(['kurtosis'])
# X = data.drop(columns=['RNA_Name'])
# y = data['RNA_Name']

# data = data.drop(columns=['kurtosis'])
# data

X = data.drop(columns=['RNA_Name'])
y = data['RNA_Name']

X_test=tdata.drop(columns=['RNA_Name'])
y_test=tdata['RNA_Name']

X_test

,AAAAAA,AAAAAC,AAAAAG,AAAAAT,AAAACA,AAAACC,AAAACG,AAAACT,AAAAGA,AAAAGC,...,TTTTCG,TTTTCT,TTTTGA,TTTTGC,TTTTGG,TTTTGT,TTTTTA,TTTTTC,TTTTTG,TTTTTT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
612,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X

,AAAAAA,AAAAAC,AAAAAG,AAAAAT,AAAACA,AAAACC,AAAACG,AAAACT,AAAAGA,AAAAGC,...,TTTTCG,TTTTCT,TTTTGA,TTTTGC,TTTTGG,TTTTGT,TTTTTA,TTTTTC,TTTTTG,TTTTTT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
y=label_encoder.fit_transform(y)
y_test=label_encoder.fit_transform(y_test)
# Split the dataset into training and testing sets
X_train=X
y_train=y

# Normalize or scale the features if necessary (optional)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



# Train Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_scaled, y_train)

# Train XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)
xgb_classifier.fit(X_train_scaled, y_train)

# Train LightGBM classifier
lgbm_classifier = lgb.LGBMClassifier(random_state=42)
lgbm_classifier.fit(X_train_scaled, y_train)

# Evaluate classifiers
rf_accuracy = rf_classifier.score(X_test_scaled, y_test)
svm_accuracy = svm_classifier.score(X_test_scaled, y_test)
xgb_accuracy = xgb_classifier.score(X_test_scaled, y_test)
lgbm_accuracy = lgbm_classifier.score(X_test_scaled, y_test)

# Display the accuracies
print("Random Forest Accuracy:", rf_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("XGBoost Accuracy:", xgb_accuracy)
print("LightGBM Accuracy:", lgbm_accuracy)
